# Working:
This the most important to know to we are going to work on this project. So for this let's see how the project will work and steps involved in this project.<br>
1. We must have input data that could be comments from different sources with labels. Like, I hate you or I love you.
2. After that we should have labels for these comments that tell us what kind of toxicity comment it was.
3. Convert these comments into Tokens or in vectors like "I love you" the token of this might be [10,400,200] or anything else.
4. Embedding will perform on this after Tokenization. This will convert the vector into again a vector that will look like [1.0,0.5,0.1] for "I", [0.2,0.9,0.5] for "love" and [0.4,0.3,0.9,1.0] for word "you" base on the attributes of the comment.
5. After completing our preprocessing we will pass this embedding vectors to deep neural network that will give us multi-binary outputs that will be checked by our labels and make decision on toxicity.
6. Then save this model into .h5 file and make an app on this model that will Gradio App which is ligt weight to deploy any deep learning model.

### 1. Import Libraries and bring data

In [1]:
import pandas as pd
import os
import numpy as np
import tensorflow as tf

Data Loading

In [2]:
!gdown --id '1ssMBUz7cVAGeDvtftjtjQOU2qSPvrFgs'

/usr/local/lib/python3.9/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1ssMBUz7cVAGeDvtftjtjQOU2qSPvrFgs
To: /content/Toxic_comment_classifier.csv
100% 68.8M/68.8M [00:01<00:00, 41.4MB/s]


In [3]:
df=pd.read_csv('/content/Toxic_comment_classifier.csv')
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [4]:
df.iloc[90]['comment_text']

"Personal attacks in Fruit Brute VfD \n\nMy apologies if I'm being to critical, but I feel that many of the comments made in the Fruit Brute VfD debate were far from reasonable.  There had to be a more diplomatic way to disagree with 's assertion on the initial sentence than don't lie, it makes you look even more juvenile.... Learn to face up to when you've goofed, it will go a long way in your life  The attacks do to his age certainly border on a personal attack.  Were Bart133 forty, sixty, or eighty, would you have included the comment on how 'juvenile' he is?\n\nI don't expect you to apologise to anyone, but I want to make it clear that I consider your comments in this VfD debate inappropriate, and I think their are many members of the community who would agree with me.   talk 06:46, 2005 Feb 7 (UTC)"

In [5]:
df[df.columns[2:]].iloc[90]

toxic            0
severe_toxic     0
obscene          0
threat           0
insult           0
identity_hate    0
Name: 90, dtype: int64

In [6]:
df[df['insult']==1].head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
42,001810bf8c45bf5f,You are gay or antisemmitian? \n\nArchangel WH...,1,0,1,0,1,1
43,00190820581d90ce,"FUCK YOUR FILTHY MOTHER IN THE ASS, DRY!",1,0,1,0,1,0
55,0020e7119b96eeeb,Stupid peace of shit stop deleting my stuff as...,1,1,1,0,1,0
56,0020fd96ed3b8c8b,=Tony Sidaway is obviously a fistfuckee. He lo...,1,0,1,0,1,0


In [7]:
df.iloc[6]['comment_text']

'COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK'

### 2. Preprocessing

In [8]:
from tensorflow.keras.layers import TextVectorization

In [9]:
#Divide data into inputs and labels
x=df['comment_text']
y=df[df.columns[2:]].values
#Values because it will be converted into array/vectors

In [10]:
x

0         Explanation\nWhy the edits made under my usern...
1         D'aww! He matches this background colour I'm s...
2         Hey man, I'm really not trying to edit war. It...
3         "\nMore\nI can't make any real suggestions on ...
4         You, sir, are my hero. Any chance you remember...
                                ...                        
159566    ":::::And for the second time of asking, when ...
159567    You should be ashamed of yourself \n\nThat is ...
159568    Spitzer \n\nUmm, theres no actual article for ...
159569    And it looks like it was actually you who put ...
159570    "\nAnd ... I really don't think you understand...
Name: comment_text, Length: 159571, dtype: object

In [11]:
y

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [12]:
max_features=50000 #The size of vacabulary
vectorize=TextVectorization(max_tokens=max_features,
                            output_mode='int',
                            output_sequence_length=1800)

In [13]:
vectorize.adapt(x.values) #provide x.values for array values

In [14]:
vectorize.get_vocabulary()

['',
 '[UNK]',
 'the',
 'to',
 'of',
 'and',
 'a',
 'you',
 'i',
 'is',
 'that',
 'in',
 'it',
 'for',
 'this',
 'not',
 'on',
 'be',
 'as',
 'have',
 'are',
 'your',
 'with',
 'if',
 'article',
 'was',
 'or',
 'but',
 'page',
 'my',
 'an',
 'from',
 'by',
 'do',
 'at',
 'about',
 'me',
 'so',
 'wikipedia',
 'can',
 'what',
 'there',
 'all',
 'has',
 'will',
 'talk',
 'please',
 'would',
 'its',
 'no',
 'one',
 'just',
 'like',
 'they',
 'he',
 'dont',
 'which',
 'any',
 'been',
 'should',
 'more',
 'we',
 'some',
 'other',
 'who',
 'see',
 'here',
 'also',
 'his',
 'think',
 'im',
 'because',
 'know',
 'how',
 'am',
 'people',
 'why',
 'edit',
 'articles',
 'only',
 'out',
 'up',
 'when',
 'were',
 'use',
 'then',
 'may',
 'time',
 'did',
 'them',
 'now',
 'being',
 'their',
 'than',
 'thanks',
 'even',
 'get',
 'make',
 'good',
 'had',
 'very',
 'information',
 'does',
 'could',
 'well',
 'want',
 'such',
 'sources',
 'way',
 'name',
 'these',
 'deletion',
 'pages',
 'first',
 'help'

In [15]:
vectorize('Hello Pakistan')

<tf.Tensor: shape=(1800,), dtype=int64, numpy=array([ 288, 2610,    0, ...,    0,    0,    0])>

In [16]:
vectorized_text=vectorize(x.values)

In [17]:
vectorized_text

<tf.Tensor: shape=(159571, 1800), dtype=int64, numpy=
array([[  645,    76,     2, ...,     0,     0,     0],
       [    1,    54,  2489, ...,     0,     0,     0],
       [  425,   441,    70, ...,     0,     0,     0],
       ...,
       [32445,  7392,   383, ...,     0,     0,     0],
       [    5,    12,   534, ...,     0,     0,     0],
       [    5,     8,   130, ...,     0,     0,     0]])>

### 2.1 Build data Pipeline by using tensorflow **slices** 

In [18]:
dataset=tf.data.Dataset.from_tensor_slices((vectorized_text,y))
dataset=dataset.cache()
dataset=dataset.shuffle(160000)
dataset=dataset.batch(16)
dataset=dataset.prefetch(8)

In [19]:
dataset.as_numpy_iterator().next()

(array([[  70,  157,   31, ...,    0,    0,    0],
        [  48,   11, 1276, ...,    0,    0,    0],
        [ 312,   48,   42, ...,    0,    0,    0],
        ...,
        [   7,   20,   30, ...,    0,    0,    0],
        [  41,  755,   29, ...,    0,    0,    0],
        [ 443, 4094, 6687, ...,    0,    0,    0]]),
 array([[0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0],
        [1, 0, 1, 0, 1, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [1, 1, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0]]))

### 2.2 Train Test split

In [20]:
train=dataset.take(int(len(dataset)*0.7))
test=dataset.skip(int(len(dataset)*0.7)).take(int(len(dataset)*0.1))
val=dataset.skip(int(len(dataset)*0.8)).take(int(len(dataset)*0.2))

In [21]:
len(dataset)
#Total batches we have 9974 each batch has 16 values

9974

In [22]:
len(train)+len(test)+len(val)

9972

In [23]:
train.as_numpy_iterator().next()

(array([[ 725,  917,  635, ...,    0,    0,    0],
        [  40,  219,    4, ...,    0,    0,    0],
        [ 281,  376,  123, ...,    0,    0,    0],
        ...,
        [2610,   46,   33, ...,    0,    0,    0],
        [  88,    7,  149, ...,    0,    0,    0],
        [  12,  196,   61, ...,    0,    0,    0]]),
 array([[0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0]]))

# 3. Building Model

In [24]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Bidirectional,Dense,Dropout,Embedding

In [79]:
model=Sequential()
model.add(Embedding(max_features+1,32))
model.add(Bidirectional(LSTM(32,activation='tanh')))
model.add(Dense(128,activation='relu'))
model.add(Dense(256,activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dense(6,activation='sigmoid'))


In [80]:
model.compile(loss='BinaryCrossentropy',optimizer='Adam',metrics=['accuracy'])

In [81]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 32)          1600032   
                                                                 
 bidirectional_1 (Bidirectio  (None, 64)               16640     
 nal)                                                            
                                                                 
 dense_4 (Dense)             (None, 128)               8320      
                                                                 
 dense_5 (Dense)             (None, 256)               33024     
                                                                 
 dense_6 (Dense)             (None, 128)               32896     
                                                                 
 dense_7 (Dense)             (None, 6)                 774       
                                                      

In [82]:
history=model.fit(train,epochs=5,validation_data=val)

Epoch 1/5
6981/6981 [==============================] - 735s 104ms/step - loss: 0.0627 - accuracy: 0.9810 - val_loss: 0.0456 - val_accuracy: 0.9945
Epoch 2/5
6981/6981 [==============================] - 623s 89ms/step - loss: 0.0466 - accuracy: 0.9907 - val_loss: 0.0423 - val_accuracy: 0.9942
Epoch 3/5
6981/6981 [==============================] - 619s 89ms/step - loss: 0.0423 - accuracy: 0.9943 - val_loss: 0.0385 - val_accuracy: 0.9934
Epoch 4/5
6981/6981 [==============================] - 614s 88ms/step - loss: 0.0387 - accuracy: 0.9878 - val_loss: 0.0350 - val_accuracy: 0.9938
Epoch 5/5
6981/6981 [==============================] - 618s 88ms/step - loss: 0.0345 - accuracy: 0.9938 - val_loss: 0.0309 - val_accuracy: 0.9943


In [83]:
history.history

{'loss': [0.06267977505922318,
  0.046592120081186295,
  0.04225881025195122,
  0.03868046775460243,
  0.03450019657611847],
 'accuracy': [0.9810109734535217,
  0.990733802318573,
  0.9942701458930969,
  0.9877793192863464,
  0.9937508702278137],
 'val_loss': [0.04558734595775604,
  0.04231460765004158,
  0.038528844714164734,
  0.034956179559230804,
  0.03092675469815731],
 'val_accuracy': [0.994546115398407,
  0.9941700100898743,
  0.9934490919113159,
  0.9937939047813416,
  0.9942640662193298]}

# 4. Making Prediction

In [84]:
text="I will kill you"
input_text=vectorize(text)

In [85]:
model.predict(np.expand_dims(input_text,0))

1/1 [==============================] - 0s 51ms/step


array([[0.9025363 , 0.05498074, 0.07709284, 0.26900575, 0.18402745,
        0.38085333]], dtype=float32)

In [86]:
df.columns[2:]

Index(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult',
       'identity_hate'],
      dtype='object')

In [87]:
#Getting data from pipelines
batch_x,batch_y=test.as_numpy_iterator().next()

In [88]:
(model.predict(batch_x)>0.5).astype('int')

1/1 [==============================] - 0s 61ms/step


array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [89]:
batch_y

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

# 5. Evaluate the model

In [90]:
from tensorflow.keras.metrics import Precision,Recall,CategoricalAccuracy

In [91]:
pre=Precision()
re=Recall()
acc=CategoricalAccuracy()

In [92]:
for batch in test.as_numpy_iterator():
  #unpack the batch
  x_true,y_true=batch
  #make a prediction
  yhat=model.predict(x_true)
  y_true=y_true.flatten()
  yhat=yhat.flatten()
  
  pre.update_state(y_true,yhat)
  re.update_state(y_true,yhat)
  acc.update_state(y_true,yhat)

1/1 [==============================] - 0s 55ms/step


In [93]:
print(f'Precision: {pre.result().numpy()}, Recall:{re.result().numpy()}, Accuracy:{acc.result().numpy()}')

Precision: 0.8560472130775452, Recall:0.8163150548934937, Accuracy:0.49548646807670593


In [103]:
#save model
model.save('comment_toxicity2.h5')

# 6. Gradio

In [95]:
!pip install gradio jinja2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [96]:
import tensorflow as tf
import gradio as gr

In [104]:
model=tf.keras.models.load_model('comment_toxicity2.h5')

In [105]:
text="I love you"
input_text=vectorize(text)
model.predict(np.expand_dims(input_text,0))

1/1 [==============================] - 1s 651ms/step


array([[1.9497929e-02, 7.8956994e-08, 1.9920166e-03, 2.5267649e-04,
        8.8890363e-04, 2.3783839e-04]], dtype=float32)

In [106]:
def score_comment(comment):
  vectorized_comment=vectorize([comment])
  results=model.predict(vectorized_comment)

  text=''
  for idx,col in enumerate(df.columns[2:]):
    text+='{}: {}\n'.format(col,results[0][idx]>0.5)

  return text

In [107]:
interface=gr.Interface(fn=score_comment,
                       inputs=gr.inputs.Textbox(lines=2,placeholder='Comment to score'),
                       outputs='text')

/usr/local/lib/python3.9/dist-packages/gradio/inputs.py:27: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.9/dist-packages/gradio/deprecation.py:40: UserWarning: `numeric` parameter is deprecated, and it has no effect
  warnings.warn(value)


In [108]:
interface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://834deeeae42d688b10.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces
